In [ ]:
import torch
import plotly.graph_objs as go
from transformers import AutoModel

def compute_and_visualize_param_difference(original_model_name, finetuned_model_name, param_name):
    """
    计算并可视化两个模型在特定参数上的差异，适用于 Qwen2 模型。
    
    参数:
        original_model_name (str): 原版 Hugging Face 模型名称。
        finetuned_model_name (str): 微调后的 Hugging Face 模型名称。
        param_name (str): 要比较的参数名称（如 'model.layers.0.self_attn.q_proj.weight'）。
    """
    # 加载原版模型和微调模型
    original_model = AutoModel.from_pretrained(original_model_name)
    finetuned_model = AutoModel.from_pretrained(finetuned_model_name)
    
    original_model.eval()
    finetuned_model.eval()
    
    # 获取特定参数
    original_param = original_model.state_dict()[param_name]
    finetuned_param = finetuned_model.state_dict()[param_name]
    
    # 计算参数差异
    param_diff = finetuned_param.detach().cpu().numpy() - original_param.detach().cpu().numpy()
    
    # 创建差异的热力图
    fig = go.Figure(data=go.Heatmap(
        z=param_diff,
        colorscale='RdBu',  # 红蓝渐变，红色代表正差异，蓝色代表负差异
        colorbar=dict(title="Difference")
    ))

    # 设置标题和标签
    fig.update_layout(
        title=f"Difference between {original_model_name} and {finetuned_model_name} - {param_name}",
        xaxis_title="Units",
        yaxis_title="Units",
        xaxis=dict(scaleanchor="y"),  # 保持横纵比例一致
        yaxis=dict(scaleanchor="x")
    )

    # 显示图像
    fig.show()

if __name__ == "__main__":
    # 示例用法
    original_model_name = "Qwen/Qwen2-7B-Instruct"
    finetuned_model_name = "path/to/your/finetuned/model"  # 替换为微调模型的路径或名称
    param_name = "model.layers.0.self_attn.q_proj.weight"  # 替换为你想要比较的参数名称
    
    compute_and_visualize_param_difference(original_model_name, finetuned_model_name, param_name)
